In [1]:
from tflearn import *
import tflearn
import tensorflow as tf
from PIL import Image
from collections import Counter
from glob import glob
import numpy as np

names = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

hdf5 not supported (please install/reinstall h5py)
Scipy not supported!


In [2]:
# Build the network and restore the trained model
imgAug = ImageAugmentation()
imgAug.add_random_flip_leftright()
imgAug.add_random_flip_updown()
imgAug.add_random_crop((28, 28))

imgPrep = ImagePreprocessing()
imgPrep.add_crop_center((28, 28))
imgPrep.add_zca_whitening

# Model constructor
def buildModel():
    network = input_data(shape=[None, 28, 28, 3], data_augmentation=imgAug, data_preprocessing=imgPrep)
    # CNN Block 1
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 2
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 3
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 3
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # FC Block 1
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='relu')
    network = batch_normalization(network)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='Adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.05)
    return network

net = buildModel()
model = tflearn.DNN(net, tensorboard_verbose=0)
model.load('./Final/model.tflearn')

(?, 28, 28, 32)
(?, 19, 19, 32)
(?, 19, 19, 64)
(?, 13, 13, 64)
(?, 13, 13, 128)
(?, 9, 9, 128)
(?, 9, 9, 160)
(?, 6, 6, 160)


In [4]:
# Load the Images
images = glob('./Web/*')
loaded = []
for i in images:
    loaded.append([Image.open(i), i])
    
# Voting Predictions - test each image ten times and select the mode
correct = 0
count = len(loaded)
for i in range(0, count):
    im = np.array(loaded[i][0].resize((28,28))) / 255.
    predicted = [model.predict([im])[0] for r in range(0, 10)] 
    p = [np.argmax(x) for x in predicted]
    c = Counter(p)
    predicted = names[c.most_common(1)[0][0]]
    filename = loaded[i][1]
    status = 'Wrong'
    if predicted in filename:
        status = 'Right'
        correct +=1
    print(c.most_common(1), predicted, filename, status )
    
print('Completed, {} out of {} correct'.format(correct, count))

([(6, 7)], 'frog', './Web/frog.jpg', 'Right')
([(9, 7)], 'truck', './Web/truck.jpg', 'Right')
([(4, 10)], 'deer', './Web/deer.jpg', 'Right')
([(5, 10)], 'dog', './Web/dog.jpg', 'Right')
([(8, 6)], 'ship', './Web/ship.jpg', 'Right')
([(7, 10)], 'horse', './Web/horse.jpg', 'Right')
([(2, 7)], 'bird', './Web/bird.jpg', 'Right')
([(0, 10)], 'airplane', './Web/airplane.jpg', 'Right')
([(3, 6)], 'cat', './Web/cat.jpg', 'Right')
([(0, 5)], 'airplane', './Web/automobile.jpg', 'Wrong')
Completed, 9 out of 10 correct
